In [8]:
import numpy as np
import polars as pl

import lightgbm as lgb
import optuna
import ConfigSpace as CS
from dehb import DEHB

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import time
import math
import warnings
warnings.filterwarnings('ignore')

In [9]:
# load dataset
dataset = pl.read_csv('../data/Train.csv', ignore_errors=True)
sbmssn = pl.read_csv('../data/Test.csv', ignore_errors=True)

dataset.head(n=5)

child_id,data_year,child_date,child_age,child_enrolment_date,child_months_enrolment,child_grant,child_years_in_programme,child_height,child_observe_attentive,child_observe_concentrated,child_observe_diligent,child_observe_interested,child_observe_total,child_gender,child_dob,child_zha,child_stunted,child_attends,child_attendance,child_languages,child_age_group,id_mn_best,prov_best,id_dc_best,dc_best,mn_best,ward_best,id_enumerator,id_facility,pqa_date,pqa_class_age,pqa_class_age_1,pqa_class_age_2,pqa_class_age_3,pqa_class_age_4,pqa_class_age_5,…,positionother,positionotherreason,sef_ind,language_match,elp_ind,gps_ind,pre_covid,ses_proxy,quintile_used,id_facility_n,id_ward_n,id_mn_n,id_dc_n,id_prov_n,language_assessment_w2,ses_cat,obs_lighting_1,obs_lighting_2,obs_lighting_3,obs_lighting_4,obs_lighting_5,obs_lighting_6,obs_lighting_8,obs_cooking_1,obs_cooking_2,obs_cooking_3,obs_cooking_4,obs_cooking_5,obs_cooking_6,obs_heating_1,obs_heating_2,obs_heating_3,obs_heating_4,obs_heating_5,obs_heating_6,obs_heating_7,target
str,f64,str,f64,str,f64,str,str,f64,str,str,str,str,f64,str,str,f64,str,str,f64,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,…,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ID_SYSJ2FM0D""",2022.0,"""2022-02-03""",59.0,null,null,null,null,null,"""Sometimes""","""Sometimes""","""Sometimes""","""Sometimes""",4.0,"""Female""","""2017-02-06""",null,null,null,null,null,"""50-59 months""","""GT421""","""GAUTENG""","""DC42""","""SEDIBENG""","""EMFULENI""",39.0,20005.0,761.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""Yes""","""Post COVID""",2.0,"""Yes""",7.0,14.0,107.0,134.0,1051.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,51.5
"""ID_J5BTFOZR3""",2019.0,null,60.163933,null,null,null,"""1st year in th…",103.0,"""Sometimes""","""Almost never""","""Sometimes""","""Often""",4.0,"""Female""",null,-1.356791,"""Normal""",null,null,null,"""60-69 months""",null,"""KWAZULU-NATAL""","""DC22""","""UMGUNGUNDLOVU""",null,null,null,458.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""No""","""No""","""Pre-COVID""",4.0,"""Yes""",24.0,null,null,367.0,1832.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,55.869999
"""ID_R00SN7AUD""",2022.0,"""2022-03-11""",69.0,null,null,null,null,108.400002,"""Often""","""Often""","""Sometimes""","""Often""",7.0,"""Male""","""2016-05-24""",-1.250863,"""Normal""",null,null,null,"""60-69 months""","""CPT""","""WESTERN CAPE""","""CPT""","""CITY OF CAPE T…","""CITY OF CAPE T…",85.0,20001.0,925.0,null,null,null,null,null,null,null,…,null,null,"""No""",1.0,"""Yes""","""Yes""","""Post COVID""",1.0,"""No""",8.0,24.0,1448.0,1448.0,3214.0,null,"""R0-110""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,47.52
"""ID_BSSK60PAZ""",2021.0,"""2021-10-13""",53.0,"""2020-01-15""",20.0,"""No""","""1st year in th…",98.099998,"""Almost always""","""Almost always""","""Sometimes""","""Often""",9.0,"""Male""","""2017-05-08""",-1.830364,"""Normal""",null,null,null,"""50-59 months""","""WC025""","""WESTERN CAPE""","""DC2""","""CAPE WINELANDS…","""BREEDE VALLEY""",18.0,2689.0,308.0,null,null,null,null,null,null,null,…,null,null,"""Yes""",1.0,"""Yes""","""Yes""","""Post COVID""",3.0,"""No""",4.0,22.0,76.0,629.0,3214.0,null,"""R291-750""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,58.599998
"""ID_IZTY6TC4D""",2021.0,"""2021-10-13""",57.0,"""2021-10-13""",0.0,null,"""2nd year in pr…",114.0,"""Almost always""","""Almost always""","""Almost always""","""Almost always""",12.0,"""Female""","""2016-12-19""",1.3292638,"""Normal""",null,null,null,"""50-59 months""","""KZN293""","""KWAZULU-NATAL""","""DC29""","""ILEMBE""","""NDWEDWE"

In [10]:
# drop some columns
deselect_col = [
    'child_id',
    'obs_materials', 'obs_handwashing', 'obs_area', 'obs_toilet',
    'obs_equipment', 'obs_safety', 'obs_hazard', 'pqa_class_age',
    'pra_groupings', 'pra_plans', 'obs_access_disability', 'pra_cohort',
    'pra_plan_4yrs', 'pra_plan_5yrs', 'pra_plan_4yrsother', 'pra_plan_5yrsother',
    'pra_qualification', 'pra_ncf_trainer', 'pra_training', 'pri_language',
    'pri_languageother', 'pri_meal', 'pri_money', 'pri_moneyother', 'pri_funding_salary', 'pri_expenseother',
    'pri_clinic_travel', 'pri_covid_awareness', 'pri_covid_precautions',
    'pri_food_type', 'pri_records', 'pri_support_provider', 'pri_support_providerother',
    'pri_fees_exceptions_other', 'pri_staff_changes_reasonsother',
    'pri_email_network_forum', 'pri_dsd_unregistered_other', 'pri_dsd_conditional_other',
    'pri_reason_register_year', 'pri_facilitiesother', 'pri_landother', 'pri_fundingother',
    'pri_qualification', 'hle_ecd_other', 'obs_water_running_none', 'obs_waterother'
]

dataset = dataset.select(pl.all().exclude(deselect_col))
sbmssn = sbmssn.select(pl.all().exclude(list(set(deselect_col) - set(['child_id']))))

In [11]:
# date columns we are going to recode
date_col = ['child_date', 'child_enrolment_date', 'child_dob', 'pqa_date', 'pra_date', 'pri_date', 'obs_date']

for cn in date_col: 
    dataset = dataset.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))
    sbmssn = sbmssn.with_columns(pl.col(cn).str.strptime(pl.Date, '%Y-%m-%d'))

for cn in date_col: 
    dataset = dataset.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.month().alias(cn + '_month').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter').cast(pl.Utf8).cast(pl.Categorical),
        pl.col(cn).dt.week().alias(cn + '_week').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).cast(pl.Int32)
    ])

    sbmssn = sbmssn.with_columns([
        pl.col(cn).dt.year().alias(cn + '_year').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.month().alias(cn + '_month').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).dt.quarter().alias(cn + '_quarter').cast(pl.Utf8).cast(pl.Categorical),
        pl.col(cn).dt.week().alias(cn + '_week').cast(pl.Utf8).cast(pl.Categorical), 
        pl.col(cn).cast(pl.Int32)
    ])

In [12]:
# Convert this to categorical
dataset = dataset.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

sbmssn = sbmssn.with_columns([
    pl.col('language_match').map_dict({0: 'No', 1: 'Yes'}, default=None).cast(pl.Categorical)
])

In [13]:
print(dataset['child_languages'].unique().to_list(), sep='\n')

['isiZulu', 'English + isiXhosa +', None, 'Sesotho', 'Afrikaans + Setswana +', 'English + Xitsonga +', 'English + isiNdebele +', 'isiZulu + isiXhosa +', 'English', 'isiXhosa + Xitsonga +', 'English + isiZulu + isiXhosa', 'English + Setswana +', 'English + isiZulu +', 'Sesotho se Leboa (Sepedi)', 'Afrikaans', 'isiXhosa', 'Tshivenda', 'English + Afrikaans +', 'Setswana', 'English + Sesotho se Leboa (Sepedi) +']


In [14]:
# extract child languages here
child_langs = ['Afrikaans', 'English', 'isiNdebele', 'isiXhosa',
               'isiZulu', 'Xitsonga', 'Setswana', 'Sesotho', 'Sepedi']

for ln in child_langs:
    dataset = dataset.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('child_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'child_languages_{ln}')
    ])

dataset = dataset.drop('child_languages')
sbmssn = sbmssn.drop('child_languages')

In [15]:
print(dataset['pri_languages'].unique().to_list(), sep='\n')

['isiZulu', 'English + Afrikaans + isiXhosa', 'Afrikaans', 'English + isiZulu +', 'Afrikaans + isiZulu +', 'Sesotho', None, 'English + isiXhosa +', 'English', 'Setswana', 'isiXhosa', 'English + Afrikaans +']


In [16]:
# extract pri_languages here
pri_langs = ['Afrikaans', 'English', 'isiXhosa', 'isiZulu', 'Setswana', 'Sesotho']

for ln in pri_langs:
    dataset = dataset.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_languages').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_languages_{ln}')
    ])

dataset = dataset.drop('pri_languages')
sbmssn = sbmssn.drop('pri_languages')

In [17]:
# fix the gps column and add polar coordinates
dataset = dataset.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

sbmssn = sbmssn.with_columns(
    pl.col('gps').str.split(' ').\
    arr.to_struct(n_field_strategy='max_width').\
    struct.rename_fields(['gps_lat', 'gps_lon'])).\
    unnest('gps')

# cast to numeric
for cn in ['gps_lat', 'gps_lon']:
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Float64))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Float64))

In [18]:
# numerics I process as categories
numcat_columns = ['id_ward', 'id_enumerator', 'id_facility', 'id_team']

for cn in numcat_columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Int64).cast(pl.Utf8).cast(pl.Categorical))

In [19]:
# replace values with NA's in train.data
dataset = dataset.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

sbmssn = sbmssn.with_columns([
    pl.when(pl.col(pl.Utf8).str.contains('Do Not Know|Missing|Don\'t know|Refuse|DON\'T KNOW|DON\'T KNOW.|DON\'T KNOW THE REASON FOR CONDITIONAL'))
    .then(None)
    .otherwise(pl.col(pl.Utf8))
    .keep_name()
])

In [20]:
# check where dataset and sbmssn diasgree on data types
for cn in dataset.select(pl.exclude('target')).columns:
    if dataset[cn].dtype != sbmssn[cn].dtype:
        print(f'Column {cn} has different data types between the two data frames.')

Column pri_network_type has different data types between the two data frames.
Column pri_staff_changes_reasons has different data types between the two data frames.
Column pri_food_donor has different data types between the two data frames.


In [21]:
# fix those columns
for answer in [1, 2, 3, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_network_type').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_network_type_{answer}')
    ])

for answer in [1, 2, 3, 4, 5, 6, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_staff_changes_reasons').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_staff_changes_reasons_{answer}')
    ])

for answer in [1, 2, 3, 4, 97]:
    dataset = dataset.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('pri_food_donor').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'pri_food_donor_{answer}')
    ])

# drop the splitted columns
dataset = dataset.drop(
    ['pri_network_type', 'pri_staff_changes_reasons', 'pri_food_donor'])
sbmssn = sbmssn.drop(
    ['pri_network_type', 'pri_staff_changes_reasons', 'pri_food_donor'])

In [22]:
# categorical encoding of some other survey responses
for answer in [1, 2, 3, 4, 5, 97]:
    dataset = dataset.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

    sbmssn = sbmssn.with_columns([
        pl.col('health').
        str.contains(ln).
        map_dict({0: 'No', 1: 'Yes'}, default=None).
        cast(pl.Categorical).
        alias(f'health_{answer}')
    ])

# drop the splitted columns
dataset = dataset.drop('health')
sbmssn = sbmssn.drop('health')

In [23]:
# other strings to categories
for cn in dataset.select(pl.col(pl.Utf8)).columns: 
    dataset = dataset.with_columns(pl.col(cn).cast(pl.Categorical))
    sbmssn = sbmssn.with_columns(pl.col(cn).cast(pl.Categorical))

In [24]:
# training - test split
train_data_x, test_data_x, train_data_y, test_data_y = train_test_split(
    dataset.drop(['target']).to_pandas(),
    dataset.get_column('target').to_pandas(),
    train_size=0.9,
    random_state=0
)

# training is splitted for tuning again
eval_data_x, valid_data_x, eval_data_y, valid_data_y = train_test_split(
    train_data_x,
    train_data_y,
    train_size=0.8,
    random_state=0
)

In [25]:
# refit model on complete training dataset and check prediction error
# FIXME

In [26]:
R = 5000/50
num_brackets = math.floor(math.log(R, 3)) + 1
print('Number of brackets', num_brackets)

Number of brackets 5


In [27]:
# define fidelity range
min_budget, max_budget = 50, 5000

# define config space
def create_search_space(seed=0):
    """Parameter space to be optimized --- contains the hyperparameters"""
    cs = CS.ConfigurationSpace(seed=seed)

    cs.add_hyperparameters([
        CS.UniformFloatHyperparameter('lambda_l1', lower=1e-4, upper=10.0, default_value=1e-4, log=True),
        CS.UniformFloatHyperparameter('lambda_l2', lower=1e-4, upper=10.0, default_value=1e-4, log=True),
        CS.UniformIntegerHyperparameter('num_leaves', lower=16, upper=256, default_value=32, log=True),
        CS.UniformFloatHyperparameter('cat_l2', lower=5.0, upper=15.0, default_value=10.0, log=False),
        CS.UniformFloatHyperparameter('cat_smooth', lower=5.0, upper=15.0, default_value=10.0, log=False),
        # 'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 1.0),
        # 'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.6, 1.0),
    ])

    return cs

cs = create_search_space(seed=0)
print(cs)

dimensions = len(cs.get_hyperparameters())
print("Dimensionality of search space: {}".format(dimensions))

Configuration space object:
  Hyperparameters:
    cat_l2, Type: UniformFloat, Range: [5.0, 15.0], Default: 10.0
    cat_smooth, Type: UniformFloat, Range: [5.0, 15.0], Default: 10.0
    lambda_l1, Type: UniformFloat, Range: [0.0001, 10.0], Default: 0.0001, on log-scale
    lambda_l2, Type: UniformFloat, Range: [0.0001, 10.0], Default: 0.0001, on log-scale
    num_leaves, Type: UniformInteger, Range: [16, 256], Default: 32, on log-scale

Dimensionality of search space: 5


In [28]:
# function interface with DEHB
def target_function(config, budget, **kwargs):
    # extracting support information
    max_budget = kwargs["max_budget"]

    if budget is None:
        budget = max_budget

    lgb_data_eval = lgb.Dataset(data=eval_data_x, label=eval_data_y)
    lgb_data_valid = lgb.Dataset(
        data=valid_data_x, label=valid_data_y, reference=lgb_data_eval)
    lgb_data_test = lgb.Dataset(
        data=test_data_x, label=test_data_y, reference=lgb_data_eval)

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'learning_rate': 0.01,
        'verbosity': -1,
        'seed': 0
    }

    params.update(config.get_dictionary())

    start = time.time()

    gbm = lgb.train(
        params=params,
        num_boost_round=int(budget),
        train_set=lgb_data_eval,
        # valid_sets=[lgb_data_valid],
        # callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)],
        # keep_training_booster=True
    )

    cost = time.time() - start

    valid_data_pred = gbm.predict(valid_data_x)
    valid_rmse = mean_squared_error(
        valid_data_y, valid_data_pred, squared=False)

    test_data_pred = gbm.predict(test_data_x)
    test_rmse = mean_squared_error(test_data_y, test_data_pred, squared=False)

    result = {
        "fitness": valid_rmse,  # DE/DEHB minimizes
        "cost": cost,
        "info": {
            "test_score": test_rmse,
            "budget": budget
        }
    }

    return result

In [7]:
# define tuning algorithm (hyper-hyperparams are default)
dehb = DEHB(
    f=target_function,
    cs=cs,
    dimensions=dimensions,
    min_budget=min_budget,
    max_budget=max_budget,
    eta=3, 
    strategy='rand1_bin',
    mutation_factor=0.5,
    crossover_prob=0.5,
    n_workers=4,
    output_path="./temp"
)

In [ ]:
# run tuning algorithm
dehb.reset()

trajectory, runtime, history = dehb.run(
    total_cost=60*30, 
    verbose=False,
    save_intermediate=False,
    max_budget=dehb.max_budget
)

In [ ]:
print(len(trajectory), len(runtime), len(history), end="\n\n")

# last recorded function evaluation
last_eval = history[-1]
config, score, cost, budget, _info = last_eval

print("Last evaluated configuration, ")
print(dehb.vector_to_configspace(config), end="")
print("got a score of {}, was evaluated at a budget of {:.2f} and "
      "took {:.3f} seconds to run.".format(score, budget, cost))
print("The additional info attached: {}".format(_info))

In [ ]:
# fit a final model to complete training set and predict submission data
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'verbosity': -1,
    'seed': 0,
}

bst = lgb.train(
    params=params,
    num_boost_round=10000,
    train_set=lgb_data_eval,
)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[639]	valid_0's rmse: 9.60753


In [ ]:
# save and load model
# bst.save_model('model.txt')
# bst = lgb.Booster(model_file='model.txt')  # init model